In [1]:
# %load imdb_data_to_graph.py
import numpy as np
import dgl
import pickle
import os
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import sys
sys.path.append('../../kg/dataloader/') 
import KGDataset
import spacy

In [2]:
dataset = KGDataset.get_dataset('../../data/kg/','wn18', '1')

In [3]:
train_edges=dataset.train
test_edges=dataset.test
valid_edges=dataset.valid
entity2id=dataset.entity2id
splits_dict={'tr':train_edges,'val':valid_edges,'test':test_edges}

In [4]:
src, etype_id, dst = train_edges
train_edges_dic={}
sm=0
dm=0
smi=100
for s, eid, d in zip(src, etype_id, dst):
    if sm<s:
        sm=s
        print(s)
    if smi>s:
        smi=s
        print(smi)
    if eid not in train_edges_dic:
        train_edges_dic[eid]=[(s,d)] 
    else:
        train_edges_dic[eid]+=[(s,d)] 

8881
35365
35723
36279
38913
40698
40791
2
40802
40912
40939
40940
1
40942
0


In [5]:
for key in train_edges_dic.keys():
    print(len(train_edges_dic[key]))

34832
34796
7382
29715
2921
1299
7402
3118
4805
2935
3116
4816
629
923
632
903
1138
80


In [6]:
wn18_dir='../../data/kg/wn18/'

In [7]:
with open(os.path.join(wn18_dir,'_word_definitions_d.pkl'), 'rb') as f:
    word_definitions_d = pickle.load(f)

In [8]:
pickle.dump(splits_dict, open(os.path.join(wn18_dir, "link_pred_splits.pickle"), "wb"),
                protocol=4);

In [9]:
train_edges_dic_dgl={('word',str(k),'word'):v for k,v in train_edges_dic.items()}

In [10]:
entity2id

{'01728840': 0,
 '09251832': 1,
 '05624042': 2,
 '02553196': 3,
 '00533897': 4,
 '10875107': 5,
 '00105554': 6,
 '10437014': 7,
 '03318438': 8,
 '09304164': 9,
 '11811473': 10,
 '01324916': 11,
 '11690455': 12,
 '06628663': 13,
 '01415393': 14,
 '06778777': 15,
 '07123552': 16,
 '00445711': 17,
 '09437454': 18,
 '13068917': 19,
 '10190516': 20,
 '11873182': 21,
 '01016778': 22,
 '12667406': 23,
 '14018918': 24,
 '07623933': 25,
 '14147627': 26,
 '01570744': 27,
 '08211924': 28,
 '00083809': 29,
 '00431552': 30,
 '14656666': 31,
 '02270404': 32,
 '02562585': 33,
 '02409508': 34,
 '08394423': 35,
 '08962187': 36,
 '00767334': 37,
 '00550117': 38,
 '05539454': 39,
 '04039381': 40,
 '11101000': 41,
 '02150306': 42,
 '04632963': 43,
 '02757462': 44,
 '14672224': 45,
 '03488438': 46,
 '08912427': 47,
 '11781430': 48,
 '07511733': 49,
 '06124395': 50,
 '09066017': 51,
 '00348801': 52,
 '13486115': 53,
 '14416473': 54,
 '01371454': 55,
 '05705722': 56,
 '04904664': 57,
 '09886540': 58,
 '12291

In [11]:
hetero_graph = dgl.heterograph(train_edges_dic_dgl)

In [12]:
word_feat = torch.tensor(np.zeros((len(word_definitions_d), len(word_definitions_d['01973759']))))

In [13]:
for k,v in word_definitions_d.items():
    word_feat[entity2id[k],:]=torch.tensor(v)

In [14]:
hetero_graph.nodes['word'].data['features'] = word_feat.float()

In [15]:
pickle.dump(hetero_graph, open(os.path.join(wn18_dir, "graph.pickle"), "wb"),
                protocol=4);

In [16]:
hetero_graph.nodes['word'].data['features']

tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.0282,  0.0381,  3.0000],
        [-0.4691,  0.5274,  0.3578,  ..., -0.0541,  0.0083,  1.0000],
        [-0.2716,  0.8514,  0.3872,  ...,  0.1837, -0.0843,  1.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0368, -0.0681,  1.0000],
        [ 0.0667,  0.1855,  0.4597,  ...,  0.0663,  0.1769,  1.0000],
        [-0.3224,  0.2966, -0.7549,  ...,  0.0869, -0.0762,  3.0000]])

In [17]:
hetero_graph_red = dgl.heterograph(train_edges_dic_dgl)
with open(os.path.join(wn18_dir,'_word_definitions_d_red.pkl'), 'rb') as f:
    word_definitions_d_red = pickle.load(f)
word_feat = torch.tensor(np.zeros((len(word_definitions_d), len(word_definitions_d_red['01973759']))))
for k,v in word_definitions_d_red.items():
    word_feat[entity2id[k],:]=torch.tensor(v)
hetero_graph_red.nodes['word'].data['features'] = word_feat.float()
pickle.dump(hetero_graph_red, open(os.path.join(wn18_dir, "graph_reduced.pickle"), "wb"),
                protocol=4);

In [18]:
pickle.dump(word_feat, open(os.path.join(wn18_dir, "features.pickle"), "wb"),
                protocol=4);

In [19]:
torch.unique(word_feat[:,-1])

tensor([0., 1., 2., 3.], dtype=torch.float64)